In [1]:
# import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import scipy.misc
import glob
import sys
import csv
import os
import random

In [2]:
# Access data
def get_data(data_dir, labels_path):
    data_filenames = glob.glob(os.path.join(data_dir, '*npy'))
    random.shuffle(data_filenames)

    num_examples = len(data_filenames)
    
    labels_dict = {}
    get_labels_dict(labels_path, labels_dict)
    
    X = np.zeros([num_examples, 151, 100])
    Y = np.zeros(num_examples)
    for i, df in enumerate(data_filenames):
        X[i, :, :] = np.load(df)
        
        label_key = df.split('/')[-1].split('.')[0].split('_')[0]
        Y[i] = labels_dict[label_key]
        
    # Convert -1 labels to 0
    Y[np.where(Y == -1)] = 0
        
    return X.astype(np.float32), Y.astype(np.float32)


def get_labels_dict(reference_path, reference):
    with open(reference_path) as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            reference[row[0]] = row[1]
            
    
def get_batch(train_path, batch_size):
    filenames = glob.glob(os.path.join(train_path, '*npy'))
    return random.sample(filenames, batch_size)

            
def generate_batch(train_path, reference_path, batch_size):
    # read in reference train/validation file
    reference_train, reference_validation = {}, {}
    get_reference(reference_path, reference_train)
#     get_reference(reference_validation_path, reference_validation)
    
    batch_train_files = get_batch(train_path, batch_size)
    
    batchX = np.zeros([batch_size, 151, 100])
    batchY = np.zeros(batch_size)
    for i, f in enumerate(batch_train_files):
        arr = np.load(f)
    
        batchX[i, :, :] = arr
        batchY[i] = reference_train[f.split('/')[-1].split('.')[0].split('_')[0]]
    
    # Convert -1 labels to 0
    batchY[np.where(batchY == -1)] = 0
    
    return batchX.astype(np.float32), batchY.astype(np.float32)

In [3]:

def _variable_on_cpu(name, shape, initializer):
    """
    Helper to create a Variable stored on CPU memory.
    """

    with tf.device('/cpu:0'):
        var = tf.get_variable(
            name=name, 
            shape=shape, 
            dtype=tf.float32, 
            initializer=initializer)

    return var

def _variable_with_weight_decay(name, shape, stddev, wd):
    """
    Helper to create an initialized Variable with weight decay.
    """

    var = _variable_on_cpu(
        name=name,
        shape=shape,
        initializer=tf.truncated_normal_initializer(
            stddev=stddev,
            dtype=tf.float32))

    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, 
                                   name='weight_loss')
        tf.add_to_collection('losses', weight_decay)

    return var


def loss(unscale_logits, labels):
    """
    Add L2Loss to all the trainable variables.
    """

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels,
        logits=unscale_logits,
        name='cross_entropy')

    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')

    tf.add_to_collection('losses', cross_entropy_mean)

    total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

    return total_loss

In [4]:
def cnn(features):

    # Input Layer.
    input_layer = tf.reshape(features["x"], [-1, 151, 100, 1])
    
        
#     input_layer = tf.map_fn(lambda input: tf.image.per_image_standardization(input), 
#                             input_layer) #Normalizes all inputs. Subtracts mean and divides by std.

    # Convolutional layer #1.
    with tf.variable_scope(name_or_scope='conv1') as scope:
        kernel = _variable_with_weight_decay(
            name='weights',
            shape=[5, 5, 1, 64],
            stddev=5e-2,
            wd=0.0)

        conv = tf.nn.conv2d(
            input=input_layer,
            filter=kernel,
            strides=[1, 1, 1, 1],
            padding='SAME')

        biases = _variable_on_cpu(
            name='biases',
            shape=[64],
            initializer=tf.constant_initializer(0.0))

        pre_activation = tf.nn.bias_add(conv, biases)

        conv1 = tf.nn.tanh(
            x=pre_activation, 
            name=scope.name)
        
        
    # Max pooling layer #1.
    pool1 = tf.nn.max_pool(
        value=conv1,
        ksize=[1, 3, 3, 1], 
        strides=[1, 2, 2, 1],
        padding='SAME', 
        name='pool1')

    
    # Local response normalization layer #1.
    norm1 = tf.nn.lrn(
        input=pool1, 
        depth_radius=4, 
        bias=1.0, 
        alpha=0.001 / 9.0, 
        beta=0.75,
        name='norm1')
    

    # Fully connected layer #1.
    with tf.variable_scope(name_or_scope='fc1') as scope:
        flat = tf.reshape(norm1, [-1, 243200])

        weights = _variable_with_weight_decay(
            name='weights',
            shape=[243200, 2],
            stddev=0.04,
            wd=0.004)

        biases = _variable_on_cpu(
            name='biases',
            shape=[2],
            initializer=tf.constant_initializer(0.1))

        pre_activation = tf.add(tf.matmul(flat, weights), biases)

        fc1 = tf.nn.relu(
            features=pre_activation,
            name=scope.name)
        
    return fc1

In [5]:

def cnn_model_fn(features, labels, mode):
    """
    Build CIFAR-10 model.
    """
    fc1 = cnn(features)
        
    # Generate predictions for PREDICT and EVAL modes.
    predictions = {
        'classes': tf.argmax(input=fc1, axis=1),
        'probabilities': tf.nn.softmax(fc1, name='softmax_tensor')
    }
    
    ####################
    # PREDICT
    ####################
    if mode == tf.estimator.ModeKeys.PREDICT: 
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions['probabilities'])
    
    else:
        # Calculate loss for both TRAIN and EVAL modes.
        labels = tf.cast(labels, tf.int64)
        total_loss = loss(fc1, labels)

        # Add summary operation for total loss visualizaiton.
        tf.summary.scalar(
            name='total_loss',
            tensor=total_loss)
    

        ####################
        # TRAIN
        ####################
        if mode == tf.estimator.ModeKeys.TRAIN:
            # Update learning rate.
    #         num_batches_per_epoch = 50000 / 128
    #         decay_steps = int(num_batches_per_epoch * 350.0)

    #         lr = tf.train.exponential_decay(
    #             learning_rate=0.1,
    #             global_step=tf.train.get_global_step(),
    #             decay_steps=decay_steps,
    #             decay_rate=0.1,
    #             staircase=True,
    #             name='learning_rate')

            # Add summary operation for learning rate visualizaiton.
    #         tf.summary.scalar(
    #             name='learning_rate', 
    #             tensor=lr)

            # Compute gradients using Gradient Descent Optimizer.
            optimizer = tf.train.AdamOptimizer()

            grads_vars = optimizer.compute_gradients(loss=total_loss)

            # Add summary operations for gradient visualizations.
            for grad, var in grads_vars:
                if grad is not None:
                    tf.summary.histogram(
                        name=var.op.name + '/gradients', 
                        values=grad)

            train_op = optimizer.minimize(
                loss=total_loss,
                global_step=tf.train.get_global_step())

            # Add evaluation metrics for TRAIN mode.
            accuracy_train = tf.metrics.accuracy(
                labels=labels, 
                predictions=predictions["classes"])

            # Add summary operation for training accuracy visualizaiton.
            tf.summary.scalar(
                name='accuracy_train',
                tensor=accuracy_train[0])

            train_summary_hook = tf.train.SummarySaverHook(
                save_steps=10,
                output_dir='models/q2-train',
                summary_op=tf.summary.merge_all())

            return tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss, 
                train_op=train_op,
                training_hooks=[train_summary_hook])

        
        ####################
        # EVALUATE
        ####################
        else:
            accuracy_valid = tf.metrics.accuracy(
                labels=labels, 
                predictions=predictions["classes"])

            # Add summary operation for validation accuracy visualizaiton.
            tf.summary.scalar(
                name='accuracy_validation',
                tensor=accuracy_valid[0])

            eval_metric_ops = {"accuracy": accuracy_valid}

            eval_summary_hook = tf.train.SummarySaverHook(
                save_steps=1,
                output_dir='models/q2-train',
                summary_op=tf.summary.merge_all())

            return tf.estimator.EstimatorSpec(
                mode=mode, 
                loss=total_loss, 
                eval_metric_ops=eval_metric_ops,
                training_hooks=[eval_summary_hook])


In [9]:
# Main function for building, training and evaluating model.
def main(train_data, train_labels, eval_data, eval_labels, test_data):
    
    estimator_dir = 'models-test-estimator/'
    train_dir = 'models-test-train/'
    
    # Delete directory containing events logs and checkpoints if it exists.
    if tf.gfile.Exists(estimator_dir):
        tf.gfile.DeleteRecursively(estimator_dir)
        
    if tf.gfile.Exists(train_dir):
        tf.gfile.DeleteRecursively(train_dir)

    # Create directory containing events logs and checkpoints.
    tf.gfile.MakeDirs(estimator_dir)
    tf.gfile.MakeDirs(train_dir)
    
    # Create the Estimator.
    cifar_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, 
        model_dir=estimator_dir)

    for _ in range(10):
        
        # Train the model.
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": train_data},
            y=train_labels,
            batch_size=128,
            num_epochs=None,
            shuffle=True)

        cifar_classifier.train(
            input_fn=train_input_fn,
            steps=100)

        # Evaluate the model and print results.
        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": eval_data},
            y=eval_labels,
            num_epochs=1,
            shuffle=False)
    
        eval_results = cifar_classifier.evaluate(input_fn=eval_input_fn)
        print(eval_results)
        
    # Generate predictions on test set.
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      num_epochs=1,
      shuffle=False)

    predictions = np.array(list(cifar_classifier.predict(input_fn=predict_input_fn))).T
    
    return predictions
        

In [10]:
train_dir = 'data/spectrogram/training/'
valid_dir = 'data/spectrogram/validation/'

X_train, Y_train = get_data(os.path.join(train_dir, 'subspec'), 
                            os.path.join(train_dir, 'REFERENCE.csv'))
                            
X_valid, Y_valid = get_data(os.path.join(valid_dir, 'subspec'), 
                            os.path.join(valid_dir, 'REFERENCE.csv'))

In [ ]:
pred = main(X_train, Y_train, X_valid, Y_valid, X_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_tf_random_seed': 1, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_session_config': None, '_model_dir': 'models-test-estimator/'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models-test-estimator/model.ckpt.
INFO:tensorflow:loss = 3.41914, step = 1
INFO:tensorflow:Saving checkpoints for 100 into models-test-estimator/model.ckpt.
INFO:tensorflow:Loss for final step: 1.70809.
INFO:tensorflow:Starting evaluation at 2017-11-18-21:09:53
INFO:tensorflow:Restoring parameters from models-test-estimator/model.ckpt-100
INFO:tensorflow:Finished evaluation at 2017-11-18-21:09:59
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.488516, global_step = 100, loss = 1.70546
{'loss': 1.7054557, 'accuracy': 0.48851591, 'global_step': 1

In [ ]:
#     #Randomly distort images if in TRAIN mode.
#     if mode == tf.estimator.ModeKeys.TRAIN:
#         input_layer = tf.map_fn(lambda input: tf.random_crop(input, [24, 24, 3]), 
#                                    input_layer) #Randomly crop image to 24 x 24.
        
#         input_layer = tf.map_fn(lambda input: tf.image.random_flip_left_right(input), 
#                                    input_layer) #Randomly flip image from left to right.
        
#         input_layer = tf.map_fn(lambda input: tf.image.random_brightness(input, max_delta=63),
#                                 input_layer)
        
#         input_layer = tf.map_fn(lambda input: tf.image.random_contrast(input, lower=0.2, upper=1.8),
#                                input_layer)
        
#         input_layer = tf.map_fn(lambda input: tf.image.per_image_standardization(input), 
#                                 input_layer) #Normalizes all inputs. Subtracts mean and divides by std.
        
#     else:        
#         input_layer = tf.map_fn(lambda input: tf.image.resize_image_with_crop_or_pad(input, 24, 24), 
#                                    input_layer) #Centrally crop image to 24 x 24.

In [ ]:
#  # Helper functions for loading data

# def get_img_array(path):
#     """
#     Given path of image, returns it's numpy array
#     """
#     return scipy.misc.imread(path)


# def get_files(folder):
#     """
#     Given path to folder, returns list of files in it
#     """
#     filenames = [file for file in glob.glob(folder+'*/*')]
#     filenames.sort()
    
#     return filenames


# def get_label(filepath, label2id):
#     """
#     Files are assumed to be labeled as: /path/to/file/999_frog.png
#     Returns label for a filepath
#     """
#     tokens = filepath.split('/')
#     label = tokens[-1].split('_')[1][:-4]
    
#     if label in label2id:
#         return label2id[label]
#     else:
#         sys.exit("Invalid label: " + label)

In [ ]:
# # Functions to load data

# def get_labels(folder, label2id):
#     """
#     Returns vector of labels extracted from filenames of all files in folder
#     :param folder: path to data folder
#     :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
#     """
#     files = get_files(folder)
#     y = []
    
#     for f in files:
#         y.append(get_label(f,label2id))
#     return np.array(y)


# def get_label_mapping(label_file):
#     """
#     Returns mappings of label to index and index to label
#     The input file has list of labels, each on a separate line.
#     """
#     with open(label_file, 'r') as f:
#         id2label = f.readlines()
#         id2label = [l.strip() for l in id2label]
#     label2id = {}
#     count = 0
    
#     for label in id2label:
#         label2id[label] = count
#         count += 1
        
#     return id2label, label2id


# def get_images(folder):
#     """
#     returns numpy array of all samples in folder
#     each column is a sample resized to 30x30 and flattened
#     """
#     files = get_files(folder)
#     images = []
#     count = 0
    
#     for f in files:
#         count += 1
#         if count % 10000 == 0:
#             print("Loaded {}/{}".format(count,len(files)))
            
#         img_arr = get_img_array(f)
                
#         img_arr = img_arr.flatten() / 255.0
#         images.append(img_arr)
        
#     X = np.column_stack(images)

#     return X

# def get_train_data(data_root_path):
#     """
#     Return X and y
#     """
#     train_data_path = data_root_path + 'train'
#     id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    
#     print(label2id)
    
#     X = get_images(train_data_path)
#     y = get_labels(train_data_path, label2id)
    
#     return X, y


# def save_predictions(filename, y):
#     """
#     Dumps y into .npy file
#     """
#     np.save(filename, y)

In [ ]:

# def _variable_on_cpu(name, shape, initializer):
#     """
#     Helper to create a Variable stored on CPU memory.
#     """

#     with tf.device('/cpu:0'):
#         var = tf.get_variable(
#             name=name, 
#             shape=shape, 
#             dtype=tf.float32, 
#             initializer=initializer)

#     return var

# def _variable_with_weight_decay(name, shape, stddev, wd):
#     """
#     Helper to create an initialized Variable with weight decay.
#     """

#     var = _variable_on_cpu(
#         name=name,
#         shape=shape,
#         initializer=tf.truncated_normal_initializer(
#             stddev=stddev,
#             dtype=tf.float32))

#     if wd is not None:
#         weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
#         tf.add_to_collection('losses', weight_decay)

#     return var


# def loss(unscale_logits, labels):
#     """
#     Add L2Loss to all the trainable variables.
#     """

#     cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
#         labels=labels,
#         logits=unscale_logits,
#         name='cross_entropy')

#     cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')

#     tf.add_to_collection('losses', cross_entropy_mean)

#     total_loss = tf.add_n(tf.get_collection('losses'), name='total_loss')

#     return total_loss


# def cnn_model_fn(features, labels, mode):
#     """
#     Build CIFAR-10 model.
#     """
#     # Input Layer.
#     input_layer = tf.reshape(features["x"], [-1, 151, 100, 1])
    
# #     #Randomly distort images if in TRAIN mode.
# #     if mode == tf.estimator.ModeKeys.TRAIN:
# #         input_layer = tf.map_fn(lambda input: tf.random_crop(input, [24, 24, 3]), 
# #                                    input_layer) #Randomly crop image to 24 x 24.
        
# #         input_layer = tf.map_fn(lambda input: tf.image.random_flip_left_right(input), 
# #                                    input_layer) #Randomly flip image from left to right.
        
# #         input_layer = tf.map_fn(lambda input: tf.image.random_brightness(input, max_delta=63),
# #                                 input_layer)
        
# #         input_layer = tf.map_fn(lambda input: tf.image.random_contrast(input, lower=0.2, upper=1.8),
# #                                input_layer)
        
# #         input_layer = tf.map_fn(lambda input: tf.image.per_image_standardization(input), 
# #                                 input_layer) #Normalizes all inputs. Subtracts mean and divides by std.
        
# #     else:        
# #         input_layer = tf.map_fn(lambda input: tf.image.resize_image_with_crop_or_pad(input, 24, 24), 
# #                                    input_layer) #Centrally crop image to 24 x 24.
        
#     input_layer = tf.map_fn(lambda input: tf.image.per_image_standardization(input), 
#                             input_layer) #Normalizes all inputs. Subtracts mean and divides by std.
    

#     # Convolutional layer #1.
#     with tf.variable_scope(name_or_scope='conv1') as scope:
#         kernel = _variable_with_weight_decay(
#             name='weights',
#             shape=[5, 5, 1, 64],
#             stddev=5e-2,
#             wd=0.0)

#         conv = tf.nn.conv2d(
#             input=input_layer,
#             filter=kernel,
#             strides=[1, 1, 1, 1],
#             padding='SAME')

#         biases = _variable_on_cpu(
#             name='biases',
#             shape=[64],
#             initializer=tf.constant_initializer(0.0))

#         pre_activation = tf.nn.bias_add(conv, biases)

#         conv1 = tf.nn.tanh(
#             x=pre_activation, 
#             name=scope.name)

#     # Max pooling layer #1.
#     pool1 = tf.nn.max_pool(
#         value=conv1,
#         ksize=[1, 3, 3, 1], 
#         strides=[1, 2, 2, 1],
#         padding='SAME', 
#         name='pool1')

#     # Local response normalization layer #1.
#     norm1 = tf.nn.lrn(
#         input=pool1, 
#         depth_radius=4, 
#         bias=1.0, 
#         alpha=0.001 / 9.0, 
#         beta=0.75,
#         name='norm1')

#     # Convolutional layer #2.
#     with tf.variable_scope(name_or_scope='conv2') as scope:
#         kernel = _variable_with_weight_decay(
#             name='weights',
#             shape=[5, 5, 64, 64],
#             stddev=5e-2,
#             wd=0.0)

#         conv = tf.nn.conv2d(
#             input=norm1,
#             filter=kernel,
#             strides=[1, 1, 1, 1],
#             padding='SAME')

#         biases = _variable_on_cpu(
#             name='biases',
#             shape=[64],
#             initializer=tf.constant_initializer(0.1))

#         pre_activation = tf.nn.bias_add(conv, biases)

#         conv2 = tf.nn.tanh(
#             x=pre_activation, 
#             name=scope.name)

#     # Local response normalization layer #2.
#     norm2 = tf.nn.lrn(
#         input=conv2, 
#         depth_radius=4, 
#         bias=1.0, 
#         alpha=0.001 / 9.0, 
#         beta=0.75,
#         name='norm2')

#     # Max pooling layer #2.
#     pool2 = tf.nn.max_pool(
#         value=norm2,
#         ksize=[1, 3, 3, 1], 
#         strides=[1, 2, 2, 1],
#         padding='SAME', 
#         name='pool2')

#     # Fully connected layer #1.
#     with tf.variable_scope(name_or_scope='fc1') as scope:
#         flat = tf.reshape(pool2, [-1, 4096])

#         weights = _variable_with_weight_decay(
#             name='weights',
#             shape=[4096, 384],
#             stddev=0.04,
#             wd=0.004)

#         biases = _variable_on_cpu(
#             name='biases',
#             shape=[384],
#             initializer=tf.constant_initializer(0.1))

#         pre_activation = tf.add(tf.matmul(flat, weights), biases)

#         fc1 = tf.nn.relu(
#             features=pre_activation,
#             name=scope.name)

#     # Fully connected layer #2.
#     with tf.variable_scope(name_or_scope='fc2') as scope:
#         weights = _variable_with_weight_decay(
#             name='weights',
#             shape=[384, 192],
#             stddev=0.04,
#             wd=0.004)

#         biases = _variable_on_cpu(
#             name='biases',
#             shape=[192],
#             initializer=tf.constant_initializer(0.1))

#         pre_activation = tf.add(tf.matmul(fc1, weights), biases)

#         fc2 = tf.nn.relu(
#             features=pre_activation,
#             name=scope.name)

#     # Unscaled logits layer #1.
#     with tf.variable_scope(name_or_scope='logits1') as scope:
#         weights = _variable_with_weight_decay(
#             name='weights',
#             shape=[192, 10],
#             stddev=1.0 / 192.0,
#             wd=0.0)

#         biases = _variable_on_cpu(
#             name='biases',
#             shape=[10],
#             initializer=tf.constant_initializer(0.0))

#         unscale_logits = tf.add(tf.matmul(fc2, weights), biases)
        
        
#     predictions = {
#         # Generate predictions for PREDICT and EVAL modes.
#         'classes': tf.argmax(input=unscale_logits, axis=1),
#         'probabilities': tf.nn.softmax(unscale_logits, name='softmax_tensor')
#     }
    
    
#     if mode == tf.estimator.ModeKeys.PREDICT: 
#         return tf.estimator.EstimatorSpec(mode=mode, 
#                                           predictions=predictions['probabilities'])
    
#     # Calculate loss for both TRAIN and EVAL modes.
#     labels = tf.cast(labels, tf.int64)
    
#     total_loss = loss(unscale_logits, labels)
    
#     # Add summary operation for total loss visualizaiton.
#     tf.summary.scalar(
#         name='total_loss',
#         tensor=total_loss)
    
#     # Configure the Training Op for TRAIN mode.
#     if mode == tf.estimator.ModeKeys.TRAIN:
#         #Update learning rate.
# #         num_batches_per_epoch = 50000 / 128
# #         decay_steps = int(num_batches_per_epoch * 350.0)
        
# #         lr = tf.train.exponential_decay(
# #             learning_rate=0.1,
# #             global_step=tf.train.get_global_step(),
# #             decay_steps=decay_steps,
# #             decay_rate=0.1,
# #             staircase=True,
# #             name='learning_rate')

#         # Add summary operation for learning rate visualizaiton.
# #         tf.summary.scalar(
# #             name='learning_rate', 
# #             tensor=lr)
        
#         # Compute gradients using Gradient Descent Optimizer.
#         optimizer = tf.train.AdamOptimizer()
        
#         grads_vars = optimizer.compute_gradients(loss=total_loss)
        
#         # Add summary operations for gradient visualizations.
#         for grad, var in grads_vars:
#             if grad is not None:
#                 tf.summary.histogram(
#                     name=var.op.name + '/gradients', 
#                     values=grad)
        
#         train_op = optimizer.minimize(
#             loss=total_loss,
#             global_step=tf.train.get_global_step())
        
#         # Add evaluation metrics for TRAIN mode.
#         accuracy_train = tf.metrics.accuracy(
#             labels=labels, 
#             predictions=predictions["classes"])

#         # Add summary operation for training accuracy visualizaiton.
#         tf.summary.scalar(
#             name='accuracy_train',
#             tensor=accuracy_train[0])
        
#         train_summary_hook = tf.train.SummarySaverHook(
#             save_steps=10,
#             output_dir='models/q2-train',
#             summary_op=tf.summary.merge_all())
        
#         return tf.estimator.EstimatorSpec(
#             mode=mode,
#             loss=total_loss, 
#             train_op=train_op,
#             training_hooks=[train_summary_hook])
    
    
#     # Add evaluation metrics for EVAL mode.
#     accuracy_valid = tf.metrics.accuracy(
#         labels=labels, 
#         predictions=predictions["classes"])
    
#     # Add summary operation for validation accuracy visualizaiton.
#     tf.summary.scalar(
#         name='accuracy_validation',
#         tensor=accuracy_valid[0])
    
#     eval_metric_ops = {"accuracy": accuracy_valid}
    
#     eval_summary_hook = tf.train.SummarySaverHook(
#         save_steps=1,
#         output_dir='models/q2-train',
#         summary_op=tf.summary.merge_all())
    
#     return tf.estimator.EstimatorSpec(
#         mode=mode, 
#         loss=total_loss, 
#         eval_metric_ops=eval_metric_ops,
#         training_hooks=[eval_summary_hook])

